# 第3章: シングルエージェントとツール活用

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### ライブラリのインストール

このセルは、LangGraphおよび関連するLangChainライブラリ、特にツール利用に必要な `langchain_community` や、検索ツール用の `duckduckgo-search` などをインストールします。

In [ ]:
# === ライブラリのインストール ===
!pip install -qU langchain langgraph langchain_openai langchain_community duckduckgo-search

# --- その他の推奨ライブラリ ---
!pip install -qU python-dotenv pygraphviz pydotplus graphviz

### OpenAI APIキーの設定

このノートブックでは、エージェントの中核となるLLMとしてOpenAIのモデルを使用します。
事前にOpenAIのAPIキーを取得し、環境変数 `OPENAI_API_KEY` に設定するか、Google Colabの場合はColabのシークレットマネージャーに `OPENAI_API_KEY` という名前でキーを登録してください。

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

try:
    from google.colab import userdata
    if "OPENAI_API_KEY" in userdata.get_all():
        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    print("OpenAI APIキーをColabシークレットからロードしました。")
except ImportError:
    if os.getenv("OPENAI_API_KEY"):
        print("OpenAI APIキーを.envファイルからロードしました。")
    else:
        print("OpenAI APIキーが見つかりません。環境変数に設定するか、Colabシークレットに登録してください。")

# LLMとToolの準備
from langchain_openai import ChatOpenAI
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

# OpenAIモデルを初期化 (ツール呼び出しに対応したモデルが望ましい)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
search_tool = DuckDuckGoSearchRun()

print(f"LLM ({llm.model_name}) と検索ツール ({search_tool.name}) の準備ができました。")

---

## 問題001: 基本的なツールの実装と ToolNode の使用

### 課題
LangGraphエージェントが外部の機能を利用するためには、「ツール」を定義し、それをグラフ内で呼び出す必要があります。この問題では、簡単な算術演算を行うツール（例: 二つの数値を足し算する）を作成し、`ToolNode` を使ってグラフに組み込み、LLMからの指示でそのツールを実行させる方法を学びます。

*   **学習内容:** `@tool`デコレータを使ったツールの定義方法、`ToolNode`の基本的な使い方、LLMにツールを使わせるためのプロンプトの工夫（限定的）、そしてツール実行結果を状態に反映させる方法を理解します。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List, Optional
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

# --- ツールの定義 ---
@tool
def add_numbers(a: int, b: int) -> int:
    """二つの整数 a と b を足し算します。"""
    print(f"Tool 'add_numbers' called with a={a}, b={b}")
    return a + b

tools = [add_numbers]
tool_node = ToolNode(tools)

# --- 状態定義 (State) ---
class BasicToolAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    # 他に必要な状態があれば追加 (例: tool_call_idなど)

# --- ノード定義 (Nodes) ---
def agent_node(state: BasicToolAgentState, config):
    # LLMにツール利用を促す (tool_choiceはまだ使わない)
    # bind_tools を使ってLLMにツール情報を渡す
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# --- ルーター関数 (ツール呼び出しがあるか判断) ---
def should_call_tool_router(state: BasicToolAgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "call_tool_node" # ToolNodeへ
    return END # ツール呼び出しがなければ終了

# --- グラフ構築 ---
workflow = StateGraph(BasicToolAgentState)
workflow.add_node("agent", agent_node)
workflow.add_node("tool_executor", tool_node) # ToolNodeを追加

workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_call_tool_router,
    {
        "call_tool_node": "tool_executor",
        END: END
    }
)
workflow.add_edge("tool_executor", "agent") # ツール実行後、再度agentノードへ戻り結果を処理

graph = workflow.compile()

# --- 実行 --- 
print("--- 基本ツールのテスト (5 + 8 の計算) ---")
initial_messages = [HumanMessage(content="5たす8を計算してください。")]
inputs = {"messages": initial_messages}
for event in graph.stream(inputs, {"recursion_limit": 5}):
    print(event)

final_state = graph.invoke(inputs, {"recursion_limit": 5})
print("\n最終的なAIの応答:")
for msg in final_state["messages"]:
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        print(msg.content)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List, Optional
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode # ToolNodeをインポート
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from IPython.display import Image, display

# --- ツールの定義 ---
@tool
def add_numbers(a: int, b: int) -> int:
    """二つの整数 a と b を足し算し、その結果を返します。"""
    print(f"Tool 'add_numbers' is called with a={a}, b={b}")
    result = a + b
    print(f"  -> Result: {result}")
    return result

tools_list = [add_numbers] # ツールのリスト
tool_executor_node = ToolNode(tools_list) # ToolNodeのインスタンス化

# --- 状態定義 (State) ---
class BasicToolAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    # ツール呼び出しIDや結果を明示的に状態に持ちたい場合はここに追加

# --- ノード定義 (Nodes) ---
def agent_decision_node(state: BasicToolAgentState, config):
    print("\nagent_decision_node: LLMに判断を仰ぎます...")
    current_messages = state["messages"]
    print(f"  -> 現在のメッセージ履歴: {current_messages}")
    
    # LLMにツール情報をバインドして、ツール利用を判断させる
    llm_with_tools = llm.bind_tools(tools_list)
    ai_response = llm_with_tools.invoke(current_messages)
    print(f"  -> LLMからの応答: {ai_response}")
    
    return {"messages": [ai_response]} # LLMの応答(AIMessage)を履歴に追加

# --- ルーター関数 (ツール呼び出しがあるか判断) ---
def should_call_tool_router(state: BasicToolAgentState):
    last_message = state["messages"][-1]
    print(f"\nshould_call_tool_router: 最後のメッセージを評価 -> {last_message}")
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        print("  -> ツール呼び出し検知。tool_executor_nodeへ")
        return "call_tool_node" # ToolNodeへ遷移するエッジ名
    print("  -> ツール呼び出しなし。ENDへ")
    return END # ツール呼び出しがなければ終了

# --- グラフ構築 ---
workflow = StateGraph(BasicToolAgentState)

workflow.add_node("agent", agent_decision_node)
workflow.add_node("tool_executor", tool_executor_node) # ToolNodeをグラフに追加

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent", # agentノードの後に分岐
    should_call_tool_router, # ルーター関数
    {
        "call_tool_node": "tool_executor", # ルーターが "call_tool_node" を返したら tool_executor へ
        END: END  # ルーターが END を返したら終了
    }
)

# ToolNodeの実行後、再度agentノードに戻ってツール実行結果を処理させる
workflow.add_edge("tool_executor", "agent") 

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
print("\n--- 基本ツールのテスト (5 + 8 の計算) ---")
initial_messages_add = [HumanMessage(content="5たす8を計算して、結果を教えてください。")]
inputs_add = {"messages": initial_messages_add}

print("\nストリーム出力 (加算):")
for i, event in enumerate(graph.stream(inputs_add, {"recursion_limit": 5})):
    print(f"Event {i+1}: {event}")

final_state_add = graph.invoke(inputs_add, {"recursion_limit": 5})
print("\n最終的なAIの応答 (加算):")
found_final_answer = False
for msg in reversed(final_state_add["messages"]):
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        print(f"  -> {msg.content}")
        assert "13" in msg.content # 5+8=13
        found_final_answer = True
        break
assert found_final_answer, "最終的なAIの応答が見つかりませんでした。"
print("アサーション成功！")
``````
</details>

## 問題002: ReAct 風エージェント - LLM によるツール使用判断と応答選択

### 課題
より自律的なエージェントは、ユーザーの質問に応じて、ツールを使うべきか、それとも直接応答すべきかをLLM自身が判断します。この問題では、検索ツール（DuckDuckGoSearchRun）を用意し、LLMが「検索が必要な質問」か「直接回答できる質問」かを判断し、必要ならツールを実行、そうでなければ直接応答する、というReAct (Reasoning and Acting) の基本的な考え方を取り入れたエージェントを構築します。

*   **学習内容:** LLMの応答にツール呼び出し(tool_calls)が含まれているかどうかで処理を分岐する方法、ツール実行結果(ToolMessage)をLLMにフィードバックして最終応答を生成させる流れを理解します。`bind_tools` を使ってLLMに利用可能なツールを認識させます。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

# search_tool は準備セルで初期化済み (DuckDuckGoSearchRun)
react_tools = [search_tool]
react_tool_node = ToolNode(react_tools)

# --- 状態定義 (State) ---
class ReActAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# --- ノード定義 (Nodes) ---
def react_agent_node(state: ReActAgentState, config):
    llm_with_tools = llm.bind_tools(react_tools)
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# --- ルーター関数 ---
def react_router(state: ReActAgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "call_tool" # ツール実行へ
    return END # 直接応答またはツール実行後の最終応答として終了

# --- グラフ構築 ---
workflow = StateGraph(ReActAgentState)
workflow.add_node("agent", react_agent_node)
workflow.add_node("tool_executor", react_tool_node)

workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    react_router,
    {"call_tool": "tool_executor", END: END}
)
workflow.add_edge("tool_executor", "agent") # ツール実行後、再度agentへ

graph = workflow.compile()

# --- 実行 --- 
print("--- ReAct風エージェントテスト (検索が必要な質問) ---")
inputs_search = {"messages": [HumanMessage(content="今日の東京の天気は？")]}
for event in graph.stream(inputs_search, {"recursion_limit": 5}): print(event)

print("\n--- ReAct風エージェントテスト (直接回答できる質問) ---")
inputs_direct = {"messages": [HumanMessage(content="こんにちは！元気ですか？")]}
for event in graph.stream(inputs_direct, {"recursion_limit": 5}): print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from IPython.display import Image, display

# search_tool (DuckDuckGoSearchRun) は準備セルで初期化済み
react_tools_list = [search_tool]
react_tool_executor_node = ToolNode(react_tools_list)

# --- 状態定義 (State) ---
class ReActAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# --- ノード定義 (Nodes) ---
def react_agent_node(state: ReActAgentState, config):
    print("\nreact_agent_node: LLMが次のアクションを決定します...")
    current_messages = state["messages"]
    print(f"  -> 現在のメッセージ: {current_messages[-1]}") # 最新のメッセージを表示
    
    # LLMにツールをバインド
    llm_with_react_tools = llm.bind_tools(react_tools_list)
    ai_response = llm_with_react_tools.invoke(current_messages)
    print(f"  -> LLMの応答: {ai_response}")
    
    return {"messages": [ai_response]}

# --- ルーター関数 (ツール呼び出しがあるか、または終了か) ---
def react_conditional_router(state: ReActAgentState):
    last_message = state["messages"][-1]
    print(f"\nreact_conditional_router: 最後のメッセージを評価 -> {last_message.pretty_repr()[:500]}...") # 長いToolMessageを考慮
    
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        print("  -> ツール呼び出し検知。tool_executorへ")
        return "call_tool_executor" # ToolNodeへ
    
    # ツール呼び出しがなく、AIMessageであれば、それが最終応答の可能性が高い
    # (ToolMessageの後に再度agentに来た場合も、最終的にtool_callsなしのAIMessageになるはず)
    if isinstance(last_message, AIMessage):
        print("  -> ツール呼び出しなしのAIMessage。ENDへ")
        return END 
    
    # 通常は上記2つのどちらかで分岐するはずだが、念のためフォールバック
    print("  -> 予期せぬ状態。暫定的にENDへ")
    return END

# --- グラフ構築 ---
workflow = StateGraph(ReActAgentState)

workflow.add_node("agent", react_agent_node)
workflow.add_node("tool_executor", react_tool_executor_node)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    react_conditional_router,
    {
        "call_tool_executor": "tool_executor",
        END: END
    }
)

# ツール実行後、再度agentノードに戻り、ツール結果を考慮して次の判断をさせる
workflow.add_edge("tool_executor", "agent")

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
print("\n--- ReAct風エージェントテスト (検索が必要な質問: 今日の東京の天気は？) ---")
inputs_search_q = {"messages": [SystemMessage(content="ユーザーの質問に答えてください。必要なら検索ツールを使って情報を調べてください。"), HumanMessage(content="今日の東京の天気は？")]}

print("\nストリーム出力 (検索質問):")
for i, event in enumerate(graph.stream(inputs_search_q, {"recursion_limit": 5})):
    print(f"Event {i+1}: {event}")

final_state_search = graph.invoke(inputs_search_q, {"recursion_limit": 5})
print("\n最終的なAIの応答 (検索質問):")
search_answer_found = False
for msg in reversed(final_state_search["messages"]):
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        print(f"  -> {msg.content}")
        search_answer_found = True
        break
assert search_answer_found, "検索質問に対する最終応答が見つかりませんでした。"

print("\n--- ReAct風エージェントテスト (直接回答できる質問: こんにちは！調子はどう？) ---")
inputs_direct_q = {"messages": [SystemMessage(content="ユーザーの質問にフレンドリーに答えてください。"), HumanMessage(content="こんにちは！調子はどう？")]}

print("\nストリーム出力 (直接質問):")
for i, event in enumerate(graph.stream(inputs_direct_q, {"recursion_limit": 5})):
    print(f"Event {i+1}: {event}")

final_state_direct = graph.invoke(inputs_direct_q, {"recursion_limit": 5})
print("\n最終的なAIの応答 (直接質問):")
direct_answer_found = False
for msg in reversed(final_state_direct["messages"]):
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        print(f"  -> {msg.content}")
        direct_answer_found = True
        break
assert direct_answer_found, "直接質問に対する最終応答が見つかりませんでした。"
print("\nアサーション成功！")
``````
</details>

## 問題003: 複数ツールからの選択

### 課題
エージェントが複数のツールを利用できる場合、状況に応じて最適なツールを選択する能力が求められます。この問題では、簡単な算術演算ツール（例: `add_numbers`）と文字列操作ツール（例: `reverse_string`）の二つを用意し、LLMがユーザーの指示内容を解釈して、どちらのツール（またはどちらも使わない）を実行すべきか判断するエージェントを構築します。

*   **学習内容:** 複数のツールをリストとしてLLMにバインドする方法、LLMの応答に含まれる`tool_calls`リストから呼び出すべきツールとその引数を特定する方法、そして選択されたツールのみを実行するロジックを理解します。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# --- ツールの定義 ---
@tool
def add(a: int, b: int) -> int:
    """整数aとbを加算します。"""
    return a + b

@tool
def reverse_string(text: str) -> str:
    """文字列textを逆順にします。"""
    return text[::-1]

multi_tools = [add, reverse_string]
multi_tool_node = ToolNode(multi_tools)

# --- 状態定義 (State) ---
class MultiToolAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# --- ノード定義 (Nodes) ---
def multi_tool_agent_node(state: MultiToolAgentState, config):
    llm_with_multi_tools = llm.bind_tools(multi_tools)
    response = llm_with_multi_tools.invoke(state["messages"])
    return {"messages": [response]}

# --- ルーター関数 ---
def multi_tool_router(state: MultiToolAgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "execute_tool"
    return END

# --- グラフ構築 ---
workflow = StateGraph(MultiToolAgentState)
workflow.add_node("agent", multi_tool_agent_node)
workflow.add_node("tool_executor", multi_tool_node)

workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    multi_tool_router,
    {"execute_tool": "tool_executor", END: END}
)
workflow.add_edge("tool_executor", "agent")

graph = workflow.compile()

# --- 実行 --- 
print("--- 複数ツール選択テスト (加算) ---")
inputs_add_test = {"messages": [HumanMessage(content="123 と 456 を足してください。")]}
for event in graph.stream(inputs_add_test, {"recursion_limit": 5}): print(event)

print("\n--- 複数ツール選択テスト (文字列逆順) ---")
inputs_reverse_test = {"messages": [HumanMessage(content="'hello world' を逆にして。")]}
for event in graph.stream(inputs_reverse_test, {"recursion_limit": 5}): print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from IPython.display import Image, display

# --- ツールの定義 ---
@tool
def add_numbers_tool(a: int, b: int) -> int:
    """二つの整数 a と b を足し算し、その結果を返します。"""
    print(f"Tool 'add_numbers_tool' called with a={a}, b={b}")
    return a + b

@tool
def reverse_string_tool(text: str) -> str:
    """与えられた文字列 text を逆順にして返します。"""
    print(f"Tool 'reverse_string_tool' called with text='{text}'")
    return text[::-1]

available_tools = [add_numbers_tool, reverse_string_tool]
selective_tool_node = ToolNode(available_tools)

# --- 状態定義 (State) ---
class MultiToolAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# --- ノード定義 (Nodes) ---
def multi_tool_agent_node(state: MultiToolAgentState, config):
    print("\nmulti_tool_agent_node: LLMがツール選択または直接応答を判断します...")
    current_messages = state["messages"]
    print(f"  -> 現在のメッセージ (最新): {current_messages[-1]}")
    
    # LLMに複数のツールをバインド
    llm_with_multiple_tools = llm.bind_tools(available_tools)
    ai_response = llm_with_multiple_tools.invoke(current_messages)
    print(f"  -> LLMの応答: {ai_response}")
    
    return {"messages": [ai_response]}

# --- ルーター関数 ---
def route_tool_or_end(state: MultiToolAgentState):
    last_message = state["messages"][-1]
    print(f"\nroute_tool_or_end: 最後のメッセージを評価 -> {last_message.pretty_repr()[:300]}...")
    
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        print("  -> ツール呼び出し検知。tool_executorへ")
        return "call_selected_tool"
    print("  -> ツール呼び出しなし。ENDへ")
    return END

# --- グラフ構築 ---
workflow = StateGraph(MultiToolAgentState)

workflow.add_node("agent", multi_tool_agent_node)
workflow.add_node("tool_executor", selective_tool_node)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    route_tool_or_end,
    {
        "call_selected_tool": "tool_executor",
        END: END
    }
)
workflow.add_edge("tool_executor", "agent") # ツール実行後、再度agentへ

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
system_prompt = SystemMessage(content="あなたはユーザーのリクエストに応じて、利用可能なツールを適切に選択して実行するか、直接応答するAIアシスタントです。")

print("\n--- 複数ツールからの選択テスト (加算タスク: 123 + 789) ---")
inputs_add_task = {"messages": [system_prompt, HumanMessage(content="123 と 789 を足した結果を教えてください。")]}
print("\nストリーム出力 (加算タスク):")
for i, event in enumerate(graph.stream(inputs_add_task, {"recursion_limit": 5})):
    print(f"Event {i+1}: {event}")
final_state_add_task = graph.invoke(inputs_add_task, {"recursion_limit": 5})
add_task_answered = False
for msg in reversed(final_state_add_task["messages"]):
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        print(f"  -> 最終応答 (加算): {msg.content}")
        assert "912" in msg.content # 123 + 789 = 912
        add_task_answered = True
        break
assert add_task_answered

print("\n--- 複数ツールからの選択テスト (文字列逆順タスク: 'LangGraph') ---")
inputs_reverse_task = {"messages": [system_prompt, HumanMessage(content="'LangGraph' という文字列を逆順にしてください。")]}
print("\nストリーム出力 (文字列逆順タスク):")
for i, event in enumerate(graph.stream(inputs_reverse_task, {"recursion_limit": 5})):
    print(f"Event {i+1}: {event}")
final_state_reverse_task = graph.invoke(inputs_reverse_task, {"recursion_limit": 5})
reverse_task_answered = False
for msg in reversed(final_state_reverse_task["messages"]):
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        print(f"  -> 最終応答 (逆順): {msg.content}")
        assert "hparGgnaL" in msg.content # LangGraph の逆
        reverse_task_answered = True
        break
assert reverse_task_answered

print("\n--- 複数ツールからの選択テスト (ツール不要な質問: こんにちは) ---")
inputs_no_tool_task = {"messages": [system_prompt, HumanMessage(content="こんにちは、元気ですか？")]}
print("\nストリーム出力 (ツール不要タスク):")
for i, event in enumerate(graph.stream(inputs_no_tool_task, {"recursion_limit": 5})):
    print(f"Event {i+1}: {event}")
final_state_no_tool_task = graph.invoke(inputs_no_tool_task, {"recursion_limit": 5})
no_tool_task_answered = False
for msg in reversed(final_state_no_tool_task["messages"]):
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        print(f"  -> 最終応答 (ツール不要): {msg.content}")
        no_tool_task_answered = True
        break
assert no_tool_task_answered
print("\n全てのアサーション成功！")
``````
</details>

## 問題004: ツール呼び出しの強制 (Forced Tool Calling)

### 課題
特定の状況下では、LLMに特定のツールを必ず使用させたい場合があります（例: ユーザーが明示的にツールの使用を指示した場合、あるいは特定の形式の情報を得るためにツールが必須の場合）。この問題では、LLMの`tool_choice`パラメータ（またはそれに類する機能、例: `ChatOpenAI`の`tool_choice`引数や`bind_tools(tool_choice=...)`）を使い、特定のツール（ここでは検索ツール）の呼び出しを強制する方法を学びます。

*   **学習内容:** LLMの`tool_choice`機能を使って、特定のツールを強制的に呼び出させる方法を理解します。これにより、エージェントの行動をより細かく制御できます。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_core.tools import tool

# --- ツールの定義 (検索ツールは準備セルで定義済み) ---
# search_tool (DuckDuckGoSearchRun)
forced_tools = [search_tool]
forced_tool_node = ToolNode(forced_tools)

# --- 状態定義 (State) ---
class ForcedToolAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# --- ノード定義 (Nodes) ---
def forced_tool_agent_node(state: ForcedToolAgentState, config):
    # tool_choice を使って特定のツール (ここでは search_tool) の使用を強制
    # llm.bind_tools の tool_choice 引数にツール名を指定
    llm_force_search = llm.bind_tools(forced_tools, tool_choice=search_tool.name)
    response = llm_force_search.invoke(state["messages"])
    return {"messages": [response]}

# --- ルーター関数 (問題002と同様) ---
def forced_tool_router(state: ForcedToolAgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "force_call_tool"
    return END

# --- グラフ構築 ---
workflow = StateGraph(ForcedToolAgentState)
workflow.add_node("agent_force_tool", forced_tool_agent_node)
workflow.add_node("tool_executor_forced", forced_tool_node)

workflow.set_entry_point("agent_force_tool")
workflow.add_conditional_edges(
    "agent_force_tool",
    forced_tool_router,
    {"force_call_tool": "tool_executor_forced", END: END}
)
workflow.add_edge("tool_executor_forced", "agent_force_tool")

graph = workflow.compile()

# --- 実行 --- 
print("--- 強制ツール呼び出しテスト (通常なら直接答えそうな質問でも検索ツールを使わせる) ---")
inputs_force = {"messages": [HumanMessage(content="こんにちは。LangGraphについて知っていることを教えて。")]}
for event in graph.stream(inputs_force, {"recursion_limit": 5}): print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_core.tools import tool # @tool デコレータ用
from IPython.display import Image, display

# --- ツールの定義 ---
# この問題では、準備セルで定義済みの search_tool (DuckDuckGoSearchRun) を使用します。
# もし別のツールを強制したい場合は、ここで定義またはインポートします。
# 例として、ダミーのカスタムツールも定義しておきます。
@tool
def get_current_time() -> str:
    """現在の時刻を文字列として返します。"""
    import datetime
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Tool 'get_current_time' called, returning: {now}")
    return now

tools_for_forcing = [search_tool, get_current_time] # LLMに知らせるツールのリスト
forced_tool_executor_node = ToolNode(tools_for_forcing) # ToolNodeは全ツールを扱えるように

# --- 状態定義 (State) ---
class ForcedToolAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# --- ノード定義 (Nodes) ---
def forced_tool_agent_node(state: ForcedToolAgentState, config):
    print("\nforced_tool_agent_node: LLMに特定のツール使用を強制します...")
    current_messages = state["messages"]
    print(f"  -> 現在のメッセージ (最新): {current_messages[-1]}")
    
    # `tool_choice`を使って特定のツール (ここでは search_tool.name) の使用を強制
    # ChatOpenAI の場合は、invokeメソッドの tool_choice 引数でも指定可能
    # llm_force_search = llm.bind_tools(tools_for_forcing, tool_choice=search_tool.name)
    # または、invoke時に渡す方法:
    # ai_response = llm.invoke(current_messages, tool_choice={"type": "function", "function": {"name": search_tool.name}})
    # より汎用的なのは bind_tools での指定
    llm_force_specific_tool = llm.bind_tools(tools_for_forcing, tool_choice=search_tool.name)
    
    ai_response = llm_force_specific_tool.invoke(current_messages)
    print(f"  -> LLMの応答 (ツール強制後): {ai_response}")
    
    # 強制されたツール呼び出しが含まれているか確認 (デバッグ用)
    if ai_response.tool_calls:
        for tc in ai_response.tool_calls:
            print(f"    - Tool call id: {tc['id']}, name: {tc['name']}, args: {tc['args']}")
            assert tc['name'] == search_tool.name, f"強制したはずの {search_tool.name} ではなく {tc['name']} が呼び出されました。"
    else:
        print("    - 警告: ツール呼び出しが強制されませんでした。LLMまたはプロンプトの確認が必要です。")
        # この場合、テストは失敗する可能性が高い

    return {"messages": [ai_response]}

# --- ルーター関数 (ツール呼び出しがあるか、または終了か) ---
def route_forced_tool_or_end(state: ForcedToolAgentState):
    last_message = state["messages"][-1]
    print(f"\nroute_forced_tool_or_end: 最後のメッセージを評価 -> {last_message.pretty_repr()[:300]}...")
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        print("  -> ツール呼び出し検知。tool_executorへ")
        return "execute_forced_tool"
    print("  -> ツール呼び出しなし。ENDへ (通常、強制後はここには来ないはず)")
    return END

# --- グラフ構築 ---
workflow = StateGraph(ForcedToolAgentState)
workflow.add_node("agent_force_tool_call", forced_tool_agent_node)
workflow.add_node("tool_executor_for_forced", forced_tool_executor_node)

workflow.set_entry_point("agent_force_tool_call")
workflow.add_conditional_edges(
    "agent_force_tool_call",
    route_forced_tool_or_end,
    {
        "execute_forced_tool": "tool_executor_for_forced",
        END: END
    }
)
workflow.add_edge("tool_executor_for_forced", "agent_force_tool_call") # ツール実行後、再度agentへ

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
print("\n--- 強制ツール呼び出しテスト --- ")
query_for_forced_search = "LangGraphの最新バージョンについて教えてください。"
print(f"質問: '{query_for_forced_search}' (通常は検索が必要だが、ここでは検索ツール使用を強制)")

inputs_force_search = {"messages": [
    SystemMessage(content="ユーザーの質問に答えるために、必ず指定されたツールを使ってください。"),
    HumanMessage(content=query_for_forced_search)
]}

print("\nストリーム出力 (強制検索):")
tool_called_in_stream = False
for i, event in enumerate(graph.stream(inputs_force_search, {"recursion_limit": 5})):
    print(f"Event {i+1}: {event}")
    # agentノードの出力でtool_callsがあるか確認
    if 'agent_force_tool_call' in event:
        agent_output_messages = event['agent_force_tool_call'].get('messages', [])
        if agent_output_messages and hasattr(agent_output_messages[-1], 'tool_calls') and agent_output_messages[-1].tool_calls:
            for tc in agent_output_messages[-1].tool_calls:
                if tc['name'] == search_tool.name:
                    tool_called_in_stream = True
                    print(f"  (ストリーム内で {search_tool.name} の呼び出しを確認)")
                    break
assert tool_called_in_stream, f"{search_tool.name} がストリーム内で呼び出されませんでした。"

final_state_forced = graph.invoke(inputs_force_search, {"recursion_limit": 5})
print("\n最終的なAIの応答 (強制検索後):")
forced_search_answered = False
for msg in reversed(final_state_forced["messages"]):
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        print(f"  -> {msg.content}")
        forced_search_answered = True
        break
assert forced_search_answered, "強制検索後の最終応答が見つかりませんでした。"
print("\nアサーション成功！")
``````
</details>

## 問題005: 複数ツールの並列実行

### 課題
LLMが一つの指示で複数のツールを同時に呼び出したい場合があります（例: 「今日の天気と最新ニュースを教えて」）。LangGraphと対応LLM（例: GPT-4o, Geminiの新しいバージョンなど）は、このような複数ツールの並列呼び出しをサポートします。この問題では、LLMが一度に複数のツール呼び出し（tool_callsに複数の要素）を生成し、それらが並列に実行され、結果がまとめてLLMに返されるフローを構築します。

*   **学習内容:** LLMが生成する`tool_calls`リストに複数のツール呼び出しが含まれる場合の処理方法、`ToolNode`がこれらをどのように並列実行するか（または逐次実行するかはToolNodeの実装や設定によるが、LangGraphは概念的に並列性をサポート）、そして複数のツール結果(ToolMessage)をどのようにLLMにフィードバックするかを理解します。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

# --- ツールの定義 (search_tool, get_current_time は問題004で定義/準備済み) ---
parallel_tools = [search_tool, get_current_time]
parallel_tool_node = ToolNode(parallel_tools) # ToolNodeはリスト内の全ツールを扱える

# --- 状態定義 (State) ---
class ParallelToolAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# --- ノード定義 (Nodes) ---
def parallel_tool_agent_node(state: ParallelToolAgentState, config):
    # LLMにツール群をバインド。LLMが複数のツールコールを返すことを期待。
    llm_with_parallel_tools = llm.bind_tools(parallel_tools)
    response = llm_with_parallel_tools.invoke(state["messages"])
    return {"messages": [response]}

# --- ルーター関数 (問題002, 004と同様) ---
def parallel_tool_router(state: ParallelToolAgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "call_parallel_tools"
    return END

# --- グラフ構築 ---
workflow = StateGraph(ParallelToolAgentState)
workflow.add_node("agent_parallel", parallel_tool_agent_node)
workflow.add_node("tool_executor_parallel", parallel_tool_node)

workflow.set_entry_point("agent_parallel")
workflow.add_conditional_edges(
    "agent_parallel",
    parallel_tool_router,
    {"call_parallel_tools": "tool_executor_parallel", END: END}
)
workflow.add_edge("tool_executor_parallel", "agent_parallel")

graph = workflow.compile()

# --- 実行 --- 
print("--- 複数ツール並列実行テスト --- ")
query_parallel = "今日の東京の天気と、現在の時刻を教えてください。"
inputs_parallel = {"messages": [HumanMessage(content=query_parallel)]}

for event in graph.stream(inputs_parallel, {"recursion_limit": 5}):
    print(event)
    # AIMessageのtool_callsに複数の呼び出しが含まれているか確認
    if 'agent_parallel' in event and event['agent_parallel']['messages'][-1].tool_calls:
        if len(event['agent_parallel']['messages'][-1].tool_calls) > 1:
            print("  -> 検知: LLMが複数のツール呼び出しを生成しました！")
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from IPython.display import Image, display

# --- ツールの定義 (search_tool, get_current_time は問題004で定義/準備済み) ---
parallel_tools_list = [search_tool, get_current_time] # 並列実行させたいツールのリスト
parallel_tool_executor_node = ToolNode(parallel_tools_list)

# --- 状態定義 (State) ---
class ParallelToolAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# --- ノード定義 (Nodes) ---
def parallel_tool_agent_node(state: ParallelToolAgentState, config):
    print("\nparallel_tool_agent_node: LLMが複数のツール使用または直接応答を判断します...")
    current_messages = state["messages"]
    print(f"  -> 現在のメッセージ (最新): {current_messages[-1]}")
    
    llm_with_parallel_capability = llm.bind_tools(parallel_tools_list)
    ai_response = llm_with_parallel_capability.invoke(current_messages)
    print(f"  -> LLMの応答: {ai_response}")
    
    # LLMが複数のツールコールを返したか確認 (デバッグ用)
    if hasattr(ai_response, 'tool_calls') and ai_response.tool_calls and len(ai_response.tool_calls) > 1:
        print(f"    -> 検知: LLMが {len(ai_response.tool_calls)} 個のツール呼び出しを並列で提案しました。")
        for tc in ai_response.tool_calls:
            print(f"      - Tool call id: {tc['id']}, name: {tc['name']}, args: {tc['args']}")
            
    return {"messages": [ai_response]}

# --- ルーター関数 (ツール呼び出しがあるか、または終了か) ---
def route_parallel_tools_or_end(state: ParallelToolAgentState):
    last_message = state["messages"][-1]
    print(f"\nroute_parallel_tools_or_end: 最後のメッセージを評価 -> {last_message.pretty_repr()[:400]}...")
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        print("  -> ツール呼び出し検知。tool_executorへ")
        return "execute_parallel_tools"
    print("  -> ツール呼び出しなし。ENDへ")
    return END

# --- グラフ構築 ---
workflow = StateGraph(ParallelToolAgentState)

workflow.add_node("agent_for_parallel_calls", parallel_tool_agent_node)
workflow.add_node("tool_executor_for_parallel", parallel_tool_executor_node)

workflow.set_entry_point("agent_for_parallel_calls")
workflow.add_conditional_edges(
    "agent_for_parallel_calls",
    route_parallel_tools_or_end,
    {
        "execute_parallel_tools": "tool_executor_for_parallel",
        END: END
    }
)
workflow.add_edge("tool_executor_for_parallel", "agent_for_parallel_calls")

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
print("\n--- 複数ツール並列実行テスト --- ")
query_for_parallel_execution = "今日の東京の天気と、現在の正確な時刻を教えてください。"
print(f"質問: '{query_for_parallel_execution}'")

inputs_for_parallel = {"messages": [
    SystemMessage(content="ユーザーの質問に答えるために、利用可能なツールを最大限活用してください。複数の情報源が一度に必要なら、それらを同時に要求できます。"),
    HumanMessage(content=query_for_parallel_execution)
]}

print("\nストリーム出力 (並列ツール呼び出し):")
num_tool_calls_generated = 0
tool_messages_received = []

for i, event in enumerate(graph.stream(inputs_for_parallel, {"recursion_limit": 5})):
    print(f"Event {i+1}: {event}")
    # agentノードの出力で複数のtool_callsがあるか確認
    if 'agent_for_parallel_calls' in event:
        agent_output_messages = event['agent_for_parallel_calls'].get('messages', [])
        if agent_output_messages and hasattr(agent_output_messages[-1], 'tool_calls') and agent_output_messages[-1].tool_calls:
            num_tool_calls_generated = len(agent_output_messages[-1].tool_calls)
            if num_tool_calls_generated > 1:
                print(f"  (ストリーム内で {num_tool_calls_generated} 個のツール呼び出しをLLMが生成したのを確認)")
    # tool_executorノードの出力で複数のToolMessageがあるか確認
    if 'tool_executor_for_parallel' in event:
        tool_executor_messages = event['tool_executor_for_parallel'].get('messages', [])
        tool_messages_received.extend([m for m in tool_executor_messages if isinstance(m, ToolMessage)])
        if len(tool_messages_received) > 1:
             print(f"  (ストリーム内で {len(tool_messages_received)} 個のツール結果メッセージを受信したのを確認)")

assert num_tool_calls_generated >= 1, "LLMがツール呼び出しを生成しませんでした。" # 理想は >=2 だがモデルによる
# モデルによっては厳密に2つにならない場合もあるため、ここでは1つ以上ツールが呼ばれたかで判定
print(f"LLMによって生成されたツールコール数: {num_tool_calls_generated}")
if num_tool_calls_generated < 2:
    print("警告: LLMが期待通り複数のツールコールを生成しませんでした。モデルやプロンプトの調整が必要かもしれません。")

final_state_parallel = graph.invoke(inputs_for_parallel, {"recursion_limit": 5})
print("\n最終的なAIの応答 (並列ツール呼び出し後):")
parallel_answered = False
for msg in reversed(final_state_parallel["messages"]):
    if isinstance(msg, AIMessage) and not msg.tool_calls:
        print(f"  -> {msg.content}")
        parallel_answered = True
        break
assert parallel_answered, "並列ツール呼び出し後の最終応答が見つかりませんでした。"
print("\nアサーション成功（少なくとも1つのツール呼び出しと最終応答を確認）。")
``````
</details>

## 問題006: シンプルな Plan-and-Execute エージェントの構築

### 課題
より複雑なタスクでは、まず計画（Plan）を立て、その計画に従ってステップを実行（Execute）するアプローチが有効です。この問題では、非常にシンプルなPlan-and-Execute型のエージェントを構築します。具体的には、以下の2つの主要ノードを持つグラフを作成します。
1.  **Plannerノード:** ユーザーの要求を受け取り、それを達成するためのステップのリスト（計画）を生成します（LLMを使用）。
2.  **Executorノード:** 計画の各ステップを順番に実行します（ここではダミーの実行とし、LLMは使わずにステップ内容をログ出力する程度）。全てのステップが完了したら終了します。

*   **学習内容:** 状態に「計画リスト」と「現在のステップ番号」を保持し、条件分岐を使って計画のステップを一つずつ実行していくループ構造を構築する方法を学びます。LLMを計画生成と実行（ここでは簡易的）の異なる役割で使う概念を理解します。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# --- 状態定義 (State) ---
class PlanExecuteState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    user_request: str
    plan: Optional[List[str]] # ステップのリスト
    current_step_index: int
    execution_log: List[str] # 各ステップの実行ログ

# --- ノード定義 (Nodes) ---
def planner_node(state: PlanExecuteState):
    request = state["user_request"]
    print(f"planner_node: Request '{request}' の計画を生成中...")
    # 実際にはLLMで計画生成。ここではダミープラン。
    # plan_str = llm.invoke(f"「{request}」を達成するためのステップを箇条書きで3つ提案してください。").content
    # generated_plan = [s.strip() for s in plan_str.split('\n') if s.strip() and s.startswith('- ')]
    # generated_plan = [s[2:] for s in generated_plan] # '- ' を除去
    generated_plan = [f"ステップ1: {request}の準備", f"ステップ2: {request}の実行", f"ステップ3: {request}の確認"]
    return {"plan": generated_plan, "current_step_index": 0, "execution_log": [], "messages": [AIMessage(content=f"計画生成完了: {generated_plan}")]}

def executor_node(state: PlanExecuteState):
    plan = state["plan"]
    idx = state["current_step_index"]
    step_description = plan[idx]
    log_entry = f"実行中 (ステップ {idx + 1}/{len(plan)}): {step_description}"
    print(f"executor_node: {log_entry}")
    current_log = state.get("execution_log", [])
    return {"current_step_index": idx + 1, "execution_log": current_log + [log_entry], "messages": [AIMessage(content=log_entry)]}

# --- ルーター関数 ---
def plan_router(state: PlanExecuteState):
    if state["plan"] and state["current_step_index"] < len(state["plan"]):
        return "execute_next_step"
    return END # 全ステップ完了

# --- グラフ構築 ---
workflow = StateGraph(PlanExecuteState)
workflow.add_node("planner", planner_node)
workflow.add_node("executor", executor_node)

workflow.set_entry_point("planner")
workflow.add_conditional_edges(
    "planner", # プランナーの後、最初の実行ステップへ（またはプランがなければ終了）
    plan_router, # プランがあるかチェック
    {"execute_next_step": "executor", END: END}
)
workflow.add_conditional_edges(
    "executor", # 各ステップ実行後、次のステップへ（または全ステップ完了なら終了）
    plan_router, 
    {"execute_next_step": "executor", END: END} # ループ
)
graph = workflow.compile()

# --- 実行 --- 
request = "美味しいコーヒーを淹れる"
inputs = {"messages": [HumanMessage(content=request)], "user_request": request}
for event in graph.stream(inputs, {"recursion_limit": 10}): print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class PlanExecuteState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages] # 主にデバッグや最終報告用
    user_request: str                           # ユーザーからの元のリクエスト
    plan: Optional[List[str]]                   # LLMが生成したステップのリスト
    current_step_index: int                     # 現在実行中の計画ステップのインデックス
    execution_log: List[str]                    # 各ステップの実行結果（またはログ）
    final_summary: Optional[str]                # 全ステップ実行後の最終サマリー

# --- ノード定義 (Nodes) ---
def initialize_plan_state_node(state: PlanExecuteState):
    user_req = state["messages"][-1].content
    print(f"initialize_plan_state_node: ユーザーリクエスト '{user_req}' を設定。")
    return {
        "user_request": user_req,
        "plan": None,
        "current_step_index": 0,
        "execution_log": [],
        "final_summary": None
    }

def planner_node(state: PlanExecuteState):
    request = state["user_request"]
    print(f"planner_node: リクエスト '{request}' に対する計画を生成中...")
    
    # LLMを使って計画を生成
    planner_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="あなたは優秀なプランナーです。ユーザーの複雑なリクエストを、実行可能なステップに分解してください。各ステップは簡潔に記述し、番号付きリストではなく、各行が1ステップとなるようにしてください。"),
        HumanMessage(content="リクエスト: {user_request}\n\n計画ステップ:")
    ])
    planner_chain = planner_prompt | llm
    response = planner_chain.invoke({"user_request": request})
    generated_plan_str = response.content.strip()
    
    # 生成された計画文字列をリストに変換
    # (例: "ステップ1\nステップ2" -> ["ステップ1", "ステップ2"])
    plan_steps = [step.strip() for step in generated_plan_str.split('\n') if step.strip()]
    
    if not plan_steps: # もしLLMが空の計画を返したら、ダミープランを設定
        print("  -> LLMが空の計画を返したため、ダミープランを使用します。")
        plan_steps = [f"'{request}'の準備作業", f"'{request}'の主要作業", f"'{request}'の完了確認"]
        
    print(f"  -> 生成された計画: {plan_steps}")
    return {"plan": plan_steps, "messages": [AIMessage(content=f"生成された計画: {plan_steps}")]}

def executor_node(state: PlanExecuteState):
    plan = state.get("plan")
    if not plan: # プランがなければ実行できない
        print("executor_node: 実行すべき計画がありません。")
        return {"execution_log": state.get("execution_log", []) + ["エラー: 計画なし"], "messages": [AIMessage(content="エラー: 計画がありませんでした。")]}
        
    idx = state.get("current_step_index", 0)
    if idx >= len(plan):
        print("executor_node: 全ての計画ステップが完了済みです。")
        return {}

    step_description = plan[idx]
    log_entry = f"実行完了 (ステップ {idx + 1}/{len(plan)}): {step_description}"
    print(f"executor_node: {log_entry}")
    
    # ここで実際にステップに対応するアクションを実行する (例: ツール呼び出し、別のLLM呼び出しなど)
    # この問題では、ログに記録するのみとする
    time.sleep(0.5) # ダミーの処理時間
    
    updated_log = state.get("execution_log", []) + [f"[成功] {step_description}"]
    return {"current_step_index": idx + 1, "execution_log": updated_log, "messages": [AIMessage(content=f"ステップ実行ログ: {log_entry}")]}

def summarize_execution_node(state: PlanExecuteState):
    log = state.get("execution_log", [])
    summary = f"計画「{state.get('user_request')}」の全 {len(log)} ステップが完了しました。\n実行ログ:\n" + "\n".join(log)
    print(f"summarize_execution_node: {summary}")
    return {"final_summary": summary, "messages": [AIMessage(content=summary)]}

# --- ルーター関数 ---
def route_after_planning(state: PlanExecuteState):
    print(f"\nroute_after_planning: 生成された計画 -> {state.get('plan')}")
    if state.get("plan") and len(state.get("plan", [])) > 0:
        print("  -> 計画あり。executorへ")
        return "start_execution"
    print("  -> 計画なしまたは空。ENDへ")
    return END # プランがなければ終了

def route_after_step_execution(state: PlanExecuteState):
    current_idx = state.get("current_step_index", 0)
    total_steps = len(state.get("plan", []))
    print(f"\nroute_after_step_execution: 現在ステップ {current_idx}/{total_steps}")
    if current_idx < total_steps:
        print("  -> 次のステップあり。executorへループ")
        return "execute_next_step"
    print("  -> 全ステップ完了。summarizerへ")
    return "all_steps_completed"

# --- グラフ構築 ---
workflow = StateGraph(PlanExecuteState)

workflow.add_node("initializer", initialize_plan_state_node)
workflow.add_node("planner", planner_node)
workflow.add_node("executor", executor_node)
workflow.add_node("summarizer", summarize_execution_node)

workflow.set_entry_point("initializer")
workflow.add_edge("initializer", "planner")

workflow.add_conditional_edges(
    "planner",
    route_after_planning,
    {"start_execution": "executor", END: END}
)
workflow.add_conditional_edges(
    "executor",
    route_after_step_execution,
    {"execute_next_step": "executor", "all_steps_completed": "summarizer"} # ループ
)
workflow.add_edge("summarizer", END)

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
user_task_request = "美味しいラーメンを作る"
print(f"\n--- Plan-and-Executeテスト (リクエスト: '{user_task_request}') ---")
inputs = {"messages": [HumanMessage(content=user_task_request)]}

print("\nストリーム出力:")
for i, event in enumerate(graph.stream(inputs, {"recursion_limit": 15})): # ステップ数に応じて調整
    print(f"Event {i+1}: {event}")

final_state_plan_exec = graph.invoke(inputs, {"recursion_limit": 15})
print("\n最終状態の確認:")
print(f"  ユーザーリクエスト: {final_state_plan_exec['user_request']}")
print(f"  生成された計画: {final_state_plan_exec['plan']}")
print(f"  実行ログの行数: {len(final_state_plan_exec['execution_log'])}")
print(f"  最終サマリー: {final_state_plan_exec['final_summary']}")
assert final_state_plan_exec['plan'] is not None and len(final_state_plan_exec['plan']) > 0
assert len(final_state_plan_exec['execution_log']) == len(final_state_plan_exec['plan'])
assert final_state_plan_exec['final_summary'] is not None
print("\nアサーション成功！")
``````
</details>